In [1]:
from jobflow import JobStore
from monty.serialization import loadfn, dumpfn
import os

output_dir = "workflow_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir,exist_ok=True)

def get_store_from_config(config_file_path:str):
    config = loadfn(config_file_path)
    return JobStore.from_dict_spec(config.get("JOB_STORE",config))

In [2]:
store = get_store_from_config("/Users/aaronkaplan/atomate2/config/jobflow.yaml")
store.connect()

In [3]:
_docs = [doc for doc in store.query({"metadata.job_info": "mof discovery", "metadata.fw_id": {"$gt": 36096}})]

In [10]:
docs = {}
for doc in _docs:
    mof_name = doc["metadata"]["MOF"]    
    if mof_name not in docs:
        docs[mof_name] = {
            "metadata": doc["metadata"].copy()
        }
    docs[mof_name][doc["name"]] = doc["output"]

for mof_name, mof_run in docs.items():

    if not (is_mof_init := mof_run.get("zeo++ input structure",{}).get("is_mof",False)):
        docs[mof_name].update({
            "has_complete_output": True,
            "is_mof": False
        })
        continue

    if not (mace_relax := mof_run.get("MACE relax")):
        docs[mof_name]["has_complete_output"] = False
        continue
    
    if not mace_relax["is_force_converged"]:
        docs[mof_name].update({
            "has_complete_output" : True,
            "is_mof": is_mof_init
        })
        continue

    if not (zeopp_final := mof_run.get("zeo++ mace-relaxed structure",{})):
        docs[mof_name].update({
            "has_complete_output" : False,
            "is_mof": is_mof_init
        })
        continue

    docs[mof_name].update({
        "has_complete_output" : True,
        "is_mof": zeopp_final.get("is_mof",False)
    })

n_tot = len(docs)
n_complete = len([mof_name for mof_name in docs if docs[mof_name]["has_complete_output"]])
n_is_mof = len([mof_name for mof_name in docs if docs[mof_name]["is_mof"]])

no_longer_mof = []
for mof_name, doc in docs.items():
    if not all(k in doc for k in ["zeo++ input structure","zeo++ mace-relaxed structure"]):
        continue
    if doc["zeo++ input structure"]["is_mof"] and not doc["zeo++ mace-relaxed structure"]:
        no_longer_mof.append(mof_name)

print(
    f"{n_complete} workflows have complete output from {n_tot} total.\n"
    f"{n_is_mof} are MOFs.\n"
)
if (n_no_longer_mof := len(no_longer_mof)) > 0:
    print(f"{len(no_longer_mof)} were MOFs prior to relaxation with MACE and now are no longer MOFs.")

dumpfn(docs,os.path.join(output_dir,"mof_discovery_docs.json.gz"))

100 workflows have complete output from 100 total.
87 are MOFs.

